In [287]:
import pandas as pd
import duckdb

In [289]:
import re

input_string = "[{value=Ministère de l'Intérieur, language=fr},{value=Ministro, language=local}]"
pattern = r'\{value=(.*?)\s*,\s*language=local\}'
matches = re.findall(pattern, input_string)

for value in matches:
    print(value)

Ministère de l'Intérieur, language=fr},{value=Ministro


In [131]:
sourcefile = "/Users/napo/Desktop/buildings_italy.parquet"
outputfile = "/Users/napo/Desktop/buildings_italy_metadata.parquet"

In [132]:
sql = """
SELECT * FROM read_parquet('%s') limit 1;
""" % (sourcefile)

In [133]:
df = duckdb.sql(sql).to_df()

In [5]:
df.columns

Index(['id', 'updatetime', 'version', 'names', 'level', 'height', 'numfloors',
       'class', 'sources', 'bbox', 'geometry', 'filename', 'type'],
      dtype='object')

In [6]:
sql = """
SELECT max(len(sources)) as maxlistsize FROM read_parquet('%s');
""" % (sourcefile)
df = duckdb.sql(sql).to_df()
df.maxlistsize.values[0]

1

In [7]:
sql = """
SELECT distinct(map_keys(names)) as knames FROM read_parquet('%s');
""" % (sourcefile)

In [35]:
sql = """
SELECT
    CASE
        WHEN EXISTS (SELECT * FROM UNNEST(map_keys(names)) AS common_key WHERE common_key = 'common') 
             AND EXISTS (SELECT * FROM UNNEST(map_keys(element_at(names, 'common'))) AS language_key WHERE language_key = 'language')
             AND element_at(element_at(names, 'common'), 'language') = 'local'
        THEN element_at(element_at(names, 'common'), 'value')
        ELSE NULL
    END AS extracted_value
FROM
    read_parquet('%s') where len(names) > 3 limit 3;
""" % (sourcefile)

In [285]:
sql = """
SELECT
    CASE
        WHEN EXISTS (
            SELECT 1
            FROM UNNEST(names) AS name_map,
                 UNNEST(name_map['common']) AS common_array
            WHERE EXISTS (
                SELECT 1
                FROM json_each(common_array)
                WHERE key = 'language' AND value = 'local'
            )
        )
        THEN (
            SELECT
                ARRAY_AGG(json.value->>'value')
            FROM UNNEST(names) AS name_map,
                 UNNEST(name_map['common']) AS common_array,
                 json_each(common_array) AS json
            WHERE json.key = 'language' AND json.value = 'local'
        )
        ELSE NULL
    END AS extracted_values
FROM
    read_parquet('%s')
WHERE
    cardinality(names) > 3
LIMIT 3;
""" % (sourcefile)

In [286]:
duckdb.sql(sql)

BinderException: Binder Error: UNNEST requires a single list as input

In [183]:
duckdb.sql(sql)

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                        flatten(element_at("names", 'common'))                                        │
│                                               map(varchar, varchar)[]                                                │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ [{value=Ministère de l'Intérieur, language=fr}, {value=وزارة الداخلية والتنمية المحلية, language=ar}, {value=وزارة…  │
│ [{value=Les Glaciers, language=local}]                                                                               │
│ [{value=Ritz, language=local}]                                                                                       │
└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [198]:
sql = """
SELECT 
    CASE
        WHEN flatten(names['common'])is not null
        THEN array_extract(names['common'],"language")
        ELSE NULL
    END AS extracted_value
FROM
    read_parquet('%s') where len(names) > 20 limit 3;
""" % (sourcefile)

In [199]:
duckdb.sql(sql)

BinderException: Binder Error: Referenced column "language" not found in FROM clause!
Candidate bindings: "read_parquet.names"

In [78]:
df = duckdb.sql(sql).to_df()


In [9]:
listname = []
for idx, row in df.iterrows():
    if len(row.knames) > 0:
        for l in range(len(row.knames)-1):
            listname.append(row.knames[l])

In [10]:
sql = """
SELECT 
    element_at(names,'common') as cicco,
    names['common'][1][1]['value'][1] as common_name, 
    names['common'][1][1]['language'][1] as language,
    names['common'][1][2]['value'][1] as common_name_2, 
    names['common'][1][2]['language'][1] as language_2,
    names['common'][1][3]['value'][1] as common_name_3, 
    names['common'][1][3]['language'][1] as language_3,
    names['common'][1][4]['value'][1] as common_name_4, 
    names['common'][1][4]['language'][1] as language_4,
    names['common'][1][5]['value'][1] as common_name_5, 
    names['common'][1][5]['language'][1] as language_5,
    names['common'][1][6]['value'][1] as common_name_6, 
    names['common'][1][6]['language'][1] as language_6,
    FROM read_parquet('%s') where language_6 is not null;
""" % (sourcefile)

In [11]:
duckdb.sql(sql)

┌──────────────────────┬──────────────────────┬──────────┬───┬────────────┬──────────────────────┬────────────┐
│        cicco         │     common_name      │ language │ … │ language_5 │    common_name_6     │ language_6 │
│ map(varchar, varch…  │       varchar        │ varchar  │   │  varchar   │       varchar        │  varchar   │
├──────────────────────┼──────────────────────┼──────────┼───┼────────────┼──────────────────────┼────────────┤
│ [[{value=Predjama …  │ Predjama Castle      │ en       │ … │ it         │ Höhlenburg Predjama  │ de         │
│ [[{value=Часовая б…  │ Часовая башня Свят…  │ ru       │ … │ hu         │ Horloĝturo           │ eo         │
│ [[{value=Bazylika …  │ Bazylika św. Jana …  │ pl       │ … │ cs         │ Базилика Сан Джова…  │ ru         │
│ [[{value=la Chiesa…  │ la Chiesa dell'Esa…  │ it       │ … │ de         │ Crkva Uzvišenja Sv…  │ local      │
│ [[{value=Этнологич…  │ Этнологический мис…  │ ru       │ … │ en         │ Musei Vaticani       │ local

In [12]:
df = duckdb.sql(sql).to_df()


In [13]:
sql = """
SELECT distinct(map_keys(sources[1])) as keys FROM read_parquet('%s');
""" % (sourcefile)

In [14]:
duckdb.sql(sql)

┌───────────────────────────────┐
│             keys              │
│           varchar[]           │
├───────────────────────────────┤
│ [dataset, property, recordId] │
└───────────────────────────────┘

In [15]:
sql = """
COPY (
SELECT 
    id,
    updatetime,
    version,
    names, 
    level, 
    height,
    numfloors,
    class,
    sources[1]['dataset'][1] as dataset,
    sources[1]['dataset'][1] as property,
    sources[1]['dataset'][1] as recordId,
    geometry
 FROM read_parquet('%s')
 ) TO '%s' (FORMAT PARQUET);
""" % (sourcefile,outputfile)

In [16]:
df = duckdb.sql(sql).to_df()

AttributeError: 'NoneType' object has no attribute 'to_df'